# Import Libraries

In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from login_info import cr_username
from login_info import cr_password
import time
from bs4 import BeautifulSoup

# Obtain Data

Create Selenium webdriver

In [6]:
driver_path = 'chromedriver/chromedriver'
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=driver_path)

Log into Consumer Reports website

In [3]:
driver.get('https://www.consumerreports.org/cro/modal-login/index.htm')
driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/form/div[1]/input').send_keys(cr_username)
driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/form/div[2]/input').send_keys(cr_password)
driver.find_element_by_xpath('/html/body/div/div/div/div/div[1]/form/div[6]/input').click()

Scrape Consume Reports vehicle classees

In [4]:
driver.get('https://www.consumerreports.org/cars/types/used/convertibles')
classes_html = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div').get_attribute('innerHTML')
vehicle_class_soup = BeautifulSoup(classes_html, 'html.parser')
vehicle_classes=['Convertibles']
for element in vehicle_class_soup.find_all('li'):
    vehicle_classes.append(element.text.replace('\n', ''))
vehicle_classes

['Convertibles',
 'Hybrids/EVs',
 'Luxury Cars',
 'Minivans',
 'Pickup Trucks',
 'Sedans',
 'Small Cars',
 'Sports Cars',
 'SUVs',
 'Wagons']

Create list of vehicle class urls

In [5]:
vehicle_class_urls=[]
for vehicle_class in vehicle_classes:
    formatted_vehicle_class = vehicle_class.replace('/', '-').replace(' ', '-').lower()
    vehicle_class_urls.append(f'https://www.consumerreports.org/cars/types/used/{formatted_vehicle_class}')
vehicle_class_urls

['https://www.consumerreports.org/cars/types/used/convertibles',
 'https://www.consumerreports.org/cars/types/used/hybrids-evs',
 'https://www.consumerreports.org/cars/types/used/luxury-cars',
 'https://www.consumerreports.org/cars/types/used/minivans',
 'https://www.consumerreports.org/cars/types/used/pickup-trucks',
 'https://www.consumerreports.org/cars/types/used/sedans',
 'https://www.consumerreports.org/cars/types/used/small-cars',
 'https://www.consumerreports.org/cars/types/used/sports-cars',
 'https://www.consumerreports.org/cars/types/used/suvs',
 'https://www.consumerreports.org/cars/types/used/wagons']

Get html for each vehicle class url with list of vehicles for that class

In [6]:
soups = []
for url in vehicle_class_urls:
    driver.get(url)
    time.sleep(1)
    table_html = driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div/div[2]/table/tbody').get_attribute('innerHTML')
    soups.append(BeautifulSoup(table_html,'html.parser'))

Create list of all Consumer Reports rated used vehicles from vehicle class urls html

In [7]:
cars_list = []
for soup in soups:
    for item in soup.find_all('tr'):
        if len(item.find_all('td', {'rowspan': '4'}))>0:
            model = item.find('td').find('div').find('div').find('div').find('a').find('span').text
            brand = item.find('td').find('div').find('div').find('div').find('a').text.replace(model, '')
            cars_list.append(brand+':'+model)

cars_list = list(set(cars_list))
cars_list[0:10]

['Ford:Explorer',
 'Subaru:Impreza',
 'Mercedes-Benz:GLC Coupe',
 'Chrysler:Voyager',
 'Pontiac:Bonneville',
 'Scion:FR-S',
 'Polestar:1',
 'Porsche:Cayenne',
 'Lincoln:Nautilus',
 'Mitsubishi:Eclipse Cross']

Create list of urls to scrape from cars list

In [28]:
reviews_urls_list = []
for car in cars_list:
    car = car.replace(' ', '-').lower()
    car_brand = car.split(':')[0]
    car_model = car.split(':')[1]
    reviews_urls_list.append(f'https://www.consumerreports.org/cars/{car_brand}/{car_model}/2019/overview/')
reviews_urls_list[0:10]

['https://www.consumerreports.org/cars/cadillac/ct4/2019/overview/',
 'https://www.consumerreports.org/cars/infiniti/fx/2019/overview/',
 'https://www.consumerreports.org/cars/acura/tl/2019/overview/',
 'https://www.consumerreports.org/cars/volvo/v60/2019/overview/',
 'https://www.consumerreports.org/cars/fiat/500l/2019/overview/',
 'https://www.consumerreports.org/cars/toyota/yaris-ia/2019/overview/',
 'https://www.consumerreports.org/cars/dodge/dakota/2019/overview/',
 'https://www.consumerreports.org/cars/jaguar/xf/2019/overview/',
 'https://www.consumerreports.org/cars/audi/tt/2019/overview/',
 'https://www.consumerreports.org/cars/toyota/mirai/2019/overview/']

Assign missing_urls and review_data variables to empty lists

In [29]:
missing_urls=[]
review_data = []

Scrape data from review urls

In [108]:
review_index = 0
for review_url in reviews_urls_list[review_index:]:
    driver.get(review_url)
    if driver.find_element_by_xpath('/html').text[0:14] == 'Page Not Found':
        missing_urls.append(review_url)
        print(f'{review_index} Bad: ', review_url)
        review_index += 1
    else:
        model_data = []
        model_data.append(review_url.split('/')[-5]+'-'+review_url.split('/')[-4])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div/div/div[1]/div[1]/a/div[1]').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div/div/div/div/div[1]/div[1]/a/div[2]/div/div/div[2]').text.strip()])
        for index in [2, 7]:
            model_data.extend([driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[{index}]/div[1]').text,
               driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[1]/div[2]/div/div[{index}]/div[2]/div/div/div/div/div[2]/span').text])
        for index in [2, 4, 6, 8, 10, 12]:
            model_data.extend([driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[{index}]/div[1]').text,
               driver.find_element_by_xpath(f'/html/body/div[7]/div[1]/div[1]/div[2]/div[2]/div[2]/div/div[{index}]/div[2]/div/div/div/div/div[2]/span').text])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[7]/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[1]').text,
            driver.find_element_by_xpath('/html/body/div[7]/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[2]/div/div/div/div/div[2]/span').text])
        time.sleep(1)
        driver.get(review_url.replace('overview/', 'ratings-specs/?pagestop'))
        time.sleep(20)
        driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/h3/div[2]').click()
        driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[7]/h3/div[2]').click()
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[3]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[3]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[7]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[2]/div/div/div[7]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[3]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[3]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[6]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[6]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[10]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[3]/div/div/div[10]/div[2]/div/div/div[2]').text.strip()])
        model_data.extend([driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[7]/div/div/div[7]/div[1]/div').text.title(),
            driver.find_element_by_xpath('/html/body/div[6]/div/div[2]/div[7]/div/div/div[7]/div[2]/div').text.strip()])
        review_data.append(model_data)    
        print(f'{review_index} Good: ', review_url)
        review_index += 1

507 Good:  https://www.consumerreports.org/cars/chevrolet/malibu/2019/overview/
508 Good:  https://www.consumerreports.org/cars/dodge/charger/2019/overview/


View scrape results

In [109]:
scrape_df = pd.DataFrame(review_data)
column_names = ['Model']
column_names.extend([scrape_df.loc[0].values[x//2*2-1] for x in range(2, len(scrape_df.loc[0].values)+1)])
scrape_df.columns = column_names
scrape_df.iloc[:, 0:15]

,Model,Reliability,Reliability,Acceleration,Acceleration,Braking,Braking,Ride,Ride,Noise,Noise,Front Seat Comfort,Front Seat Comfort,Rear Seat Comfort,Rear Seat Comfort
0,volvo-v60,Reliability,1 / 5,Acceleration,NA,Braking,NA,Ride,NA,Noise,NA,Front Seat Comfort,NA,Rear Seat Comfort,NA
1,fiat-500l,Reliability,NA,Acceleration,3 / 5,Braking,4 / 5,Ride,2 / 5,Noise,3 / 5,Front Seat Comfort,3 / 5,Rear Seat Comfort,4 / 5
2,jaguar-xf,Reliability,NA,Acceleration,5 / 5,Braking,5 / 5,Ride,5 / 5,Noise,4 / 5,Front Seat Comfort,5 / 5,Rear Seat Comfort,4 / 5
3,audi-tt,Reliability,NA,Acceleration,5 / 5,Braking,5 / 5,Ride,3 / 5,Noise,3 / 5,Front Seat Comfort,5 / 5,Rear Seat Comfort,1 / 5
4,toyota-mirai,Reliability,NA,Acceleration,3 / 5,Braking,3 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,4 / 5,Rear Seat Comfort,3 / 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,lexus-es,Reliability,3 / 5,Acceleration,4 / 5,Braking,4 / 5,Ride,5 / 5,Noise,5 / 5,Front Seat Comfort,5 / 5,Rear Seat Comfort,3 / 5
291,alfa-romeo-stelvio,Reliability,NA,Acceleration,4 / 5,Braking,4 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,3 / 5,Rear Seat Comfort,3 / 5
292,volkswagen-jetta,Reliability,3 / 5,Acceleration,3 / 5,Braking,4 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,4 / 5,Rear Seat Comfort,4 / 5
293,chevrolet-malibu,Reliability,NA,Acceleration,4 / 5,Braking,4 / 5,Ride,4 / 5,Noise,4 / 5,Front Seat Comfort,3 / 5,Rear Seat Comfort,4 / 5


In [110]:
scrape_df.iloc[:, 15:]

,Interior Fit and Finish,Interior Fit and Finish,Trunk/Cargo Area,Trunk/Cargo Area,Fuel Economy,Fuel Economy,Routine Handling,Routine Handling,Headlights,Headlights,Driving Position,Driving Position,Third Seat Comfort,Third Seat Comfort,Usability,Usability,Max. Load,Max. Load
0,Interior Fit and Finish,NA,Trunk/Cargo Area,NA,Fuel Economy,NA,Routine Handling,NA,Headlights,NA,Driving Position,NA,Third Seat Comfort,NA,Usability,NA,Max. Load,NA
1,Interior Fit and Finish,3 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,4 / 5,Routine Handling,4 / 5,Headlights,3 / 5,Driving Position,3 / 5,Third Seat Comfort,NA,Usability,3 / 5,Max. Load,860 lb.
2,Interior Fit and Finish,5 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,3 / 5,Routine Handling,5 / 5,Headlights,3 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,3 / 5,Max. Load,960 lb.
3,Interior Fit and Finish,5 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,4 / 5,Routine Handling,5 / 5,Headlights,2 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,2 / 5,Max. Load,770 lb.
4,Interior Fit and Finish,4 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,5 / 5,Routine Handling,2 / 5,Headlights,4 / 5,Driving Position,3 / 5,Third Seat Comfort,NA,Usability,3 / 5,Max. Load,690 lb.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,Interior Fit and Finish,5 / 5,Trunk/Cargo Area,3 / 5,Fuel Economy,4 / 5,Routine Handling,4 / 5,Headlights,2 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,2 / 5,Max. Load,905 lb.
291,Interior Fit and Finish,4 / 5,Trunk/Cargo Area,2 / 5,Fuel Economy,3 / 5,Routine Handling,5 / 5,Headlights,4 / 5,Driving Position,3 / 5,Third Seat Comfort,NA,Usability,2 / 5,Max. Load,990 lb.
292,Interior Fit and Finish,3 / 5,Trunk/Cargo Area,4 / 5,Fuel Economy,5 / 5,Routine Handling,4 / 5,Headlights,2 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,5 / 5,Max. Load,970 lb.
293,Interior Fit and Finish,3 / 5,Trunk/Cargo Area,4 / 5,Fuel Economy,4 / 5,Routine Handling,4 / 5,Headlights,4 / 5,Driving Position,4 / 5,Third Seat Comfort,NA,Usability,5 / 5,Max. Load,900 lb.


Save scrape results to csv file

In [111]:
scrape_df.to_csv('Data/Ratings.csv')

# Create Vehicle Class List

Create list of vehicle classes

In [76]:
class_list = []
for num in range(0, len(soups)):
    for item in soups[num].find_all('tr'):
        if len(item.find_all('td', {'rowspan': '4'}))>0:
            model = item.find('td').find('div').find('div').find('div').find('a').find('span').text
            brand = item.find('td').find('div').find('div').find('div').find('a').text.replace(model, '')
            class_list.append(vehicle_classes[num].split('/')[-1].title()+':'+brand.lower()+':'+model.lower())

class_list = list(set(class_list))
class_list[0:10]

['Luxury Cars:land rover:lr2',
 'Small Cars:saturn:astra',
 'Sedans:volkswagen:passat',
 'Suvs:land rover:discovery sport',
 'Wagons:fiat:500l',
 'Sedans:hyundai:azera',
 'Sports Cars:bmw:i8',
 'Luxury Cars:hyundai:equus',
 'Luxury Cars:cadillac:elr',
 'Sedans:fisker:karma']

Create df with vehicle classes

In [77]:
vehicle_class_list = []
for vehicle in class_list:
    vehicle_class_list.append(vehicle.split(':'))
vehicle_class_list[0:10]

[['Luxury Cars', 'land rover', 'lr2'],
 ['Small Cars', 'saturn', 'astra'],
 ['Sedans', 'volkswagen', 'passat'],
 ['Suvs', 'land rover', 'discovery sport'],
 ['Wagons', 'fiat', '500l'],
 ['Sedans', 'hyundai', 'azera'],
 ['Sports Cars', 'bmw', 'i8'],
 ['Luxury Cars', 'hyundai', 'equus'],
 ['Luxury Cars', 'cadillac', 'elr'],
 ['Sedans', 'fisker', 'karma']]

Create df with vehicle classes

In [78]:
vehicle_class_df = pd.DataFrame(vehicle_class_list, columns=['Class', 'Brand', 'Model'])
vehicle_class_df['Model'] = vehicle_class_df['Model'].apply(lambda x: x.replace('-', ' '))
vehicle_class_df.head()

,Class,Brand,Model
0,Luxury Cars,land rover,lr2
1,Small Cars,saturn,astra
2,Sedans,volkswagen,passat
3,Suvs,land rover,discovery sport
4,Wagons,fiat,500l


Save vehicle_class_df to csv file

In [79]:
vehicle_class_df.to_csv('Data/Vehicle_Classes.csv', index=False)

# Scrape Price Data

Create list of vehicle models to scrape data for

In [20]:
price_models_df = pd.read_csv('Data/Ratings.csv')
price_models_df['Model'] = price_models_df['Model'].apply(lambda x : x.replace('mercedes-benz', 'mercedes_benz'))
price_models_df['Model'] = price_models_df['Model'].apply(lambda x : x.replace('land-rover', 'land_rover'))
price_models_df['Model'] = price_models_df['Model'].apply(lambda x : x.replace('alfa-romeo', 'alfa_romeo'))
price_models_df['Brand'] = price_models_df['Model'].apply(lambda x: x.split('-')[0])
def unit_name(row):
    return row['Model'].replace(row['Brand']+'-', '')
price_models_df['Unit'] = price_models_df.apply(unit_name, axis=1)
price_models_df['Unit'] = price_models_df['Unit'].apply(lambda x: x.replace('-', '_'))
price_models_df.head()

,Unnamed: 0,Model,Reliability,Reliability.1,Acceleration,Acceleration.1,Braking,Braking.1,Ride,Ride.1,...,Driving Position,Driving Position.1,Third Seat Comfort,Third Seat Comfort.1,Usability,Usability.1,Max. Load,Max. Load.1,Brand,Unit
0,0,volvo-v60,Reliability,1 / 5,Acceleration,NaN,Braking,NaN,Ride,NaN,...,Driving Position,NaN,Third Seat Comfort,NaN,Usability,NaN,Max. Load,NaN,volvo,v60
1,1,fiat-500l,Reliability,NaN,Acceleration,3 / 5,Braking,4 / 5,Ride,2 / 5,...,Driving Position,3 / 5,Third Seat Comfort,NaN,Usability,3 / 5,Max. Load,860 lb.,fiat,500l
2,2,jaguar-xf,Reliability,NaN,Acceleration,5 / 5,Braking,5 / 5,Ride,5 / 5,...,Driving Position,4 / 5,Third Seat Comfort,NaN,Usability,3 / 5,Max. Load,960 lb.,jaguar,xf
3,3,audi-tt,Reliability,NaN,Acceleration,5 / 5,Braking,5 / 5,Ride,3 / 5,...,Driving Position,4 / 5,Third Seat Comfort,NaN,Usability,2 / 5,Max. Load,770 lb.,audi,tt
4,4,toyota-mirai,Reliability,NaN,Acceleration,3 / 5,Braking,3 / 5,Ride,4 / 5,...,Driving Position,3 / 5,Third Seat Comfort,NaN,Usability,3 / 5,Max. Load,690 lb.,toyota,mirai


Create list of price urls to scrape

In [4]:
price_urls = 'https://www.cars.com/research/' + price_models_df['Brand'] + '-' + price_models_df['Unit'] + '-2019/'
price_urls[0:10]

0        https://www.cars.com/research/volvo-v60-2019/
1        https://www.cars.com/research/fiat-500l-2019/
2        https://www.cars.com/research/jaguar-xf-2019/
3          https://www.cars.com/research/audi-tt-2019/
4     https://www.cars.com/research/toyota-mirai-2019/
5          https://www.cars.com/research/mazda-6-2019/
6     https://www.cars.com/research/buick-encore-2019/
7          https://www.cars.com/research/audi-a3-2019/
8    https://www.cars.com/research/ram-1500_classic...
9    https://www.cars.com/research/mercedes_benz-sl...
dtype: object

Create list of vehicle prices

In [5]:
vehicle_prices = []

Scrape vehicle price urls

In [15]:
price_index = 241
for price_url in price_urls[price_index:]:
    price_url = price_url.replace('mazda-6', 'mazda-mazda6')
    price_url = price_url.replace('mercedes_benz-sl', 'mercedes_benz-sl_450')
    price_url = price_url.replace('mercedes_benz-gle_coupe', 'mercedes_benz-amg_gle_43')
    price_url = price_url.replace('lexus-ls', 'lexus-ls_500')
    price_url = price_url.replace('toyota-yaris-2019', 'toyota-yaris-2018')
    price_url = price_url.replace('smart-fortwo', 'smart-eq_fortwo')
    price_url = price_url.replace('bmw-8_series', 'bmw-m850')
    price_url = price_url.replace('lexus-gs', 'lexus-gs_300')
    price_url = price_url.replace('chevrolet-silverado_3500hd', 'chevrolet-silverado_3500')
    price_url = price_url.replace('mini-cooper', 'mini-hardtop')
    price_url = price_url.replace('mercedes_benz-gla', 'mercedes_benz-gla_250')
    price_url = price_url.replace('gmc-sierra_2500hd', 'gmc-sierra_2500')
    price_url = price_url.replace('mercedes_benz-sprinter', 'mercedes_benz-sprinter_1500')
    price_url = price_url.replace('lexus-rx_l', 'lexus-rx_350l')
    price_url = price_url.replace('buick-regal', 'buick-regal_sportback')
    price_url = price_url.replace('bmw-3_series', 'bmw-330')
    price_url = price_url.replace('nissan-z', 'nissan-370z')
    price_url = price_url.replace('lexus-nx', 'lexus-nx_300')
    price_url = price_url.replace('mazda-3', 'mazda-mazda3')
    price_url = price_url.replace('hyundai-ioniq', 'hyundai-ioniq_hybrid')
    price_url = price_url.replace('mercedes_benz-cla', 'mercedes_benz-cla_250')
    price_url = price_url.replace('bmw-5_series', 'bmw-530')
    price_url = price_url.replace('chevrolet-bolt', 'chevrolet-bolt_ev')
    price_url = price_url.replace('lexus-lx', 'lexus-lx_570')
    price_url = price_url.replace('mercedes_benz-gle', 'mercedes_benz-gle_400')
    price_url = price_url.replace('audi-allroad', 'audi-a4_allroad')
    price_url = price_url.replace('mercedes_benz-sl_450c', 'mercedes_benz-sl_450')
    price_url = price_url.replace('bmw-4_series', 'bmw-430')
    price_url = price_url.replace('audi-r8-2019', 'audi-r8-2018')
    price_url = price_url.replace('hyundai-kona_electric', 'hyundai-kona_ev')
    price_url = price_url.replace('bmw-6_series', 'bmw-640_gran_coupe')
    price_url = price_url.replace('chevrolet-silverado_2500hd', 'chevrolet-silverado_2500')
    price_url = price_url.replace('lexus-ux', 'lexus-ux_200')
    price_url = price_url.replace('volkswagen-gti', 'volkswagen-golf_gti')
    price_url = price_url.replace('mercedes_benz-cls', 'mercedes_benz-cls_450')
    price_url = price_url.replace('honda-clarity', 'honda-clarity_plug_in_hybrid')
    price_url = price_url.replace('mercedes_benz-glc', 'mercedes_benz-glc_300')
    price_url = price_url.replace('lexus-gx', 'lexus-gx_460')
    price_url = price_url.replace('lexus-lc', 'lexus-lc_500')
    price_url = price_url.replace('gmc-sierra_3500hd', 'gmc-sierra_3500')
    price_url = price_url.replace('ford-transit', 'ford-transit_150')
    price_url = price_url.replace('alfa_romeo-4c', 'alfa_romeo-4c_spider')
    price_url = price_url.replace('lexus-is', 'lexus-is_300')
    price_url = price_url.replace('fiat-124', 'fiat-124_spider')
    price_url = price_url.replace('mini-hardtop_clubman', 'mini-clubman')
    price_url = price_url.replace('mercedes_benz-glc_300_coupe', 'mercedes_benz-glc_300')
    price_url = price_url.replace('kia-niro_electric', 'kia-niro_ev')
    price_url = price_url.replace('mercedes_benz-gls', 'mercedes_benz-gls_450')
    price_url = price_url.replace('ford-transit_150_connect', 'ford-transit_connect')
    price_url = price_url.replace('bentley-flying_spur-2019', 'bentley-flying_spur-2018')
    price_url = price_url.replace('mini-hardtop_countryman', 'mini-countryman')
    price_url = price_url.replace('lexus-rc', 'lexus-rc_300')
    price_url = price_url.replace('lexus-rx-2019', 'lexus-rx_350-2019')
    price_url = price_url.replace('bmw-7_series', 'bmw-740')
    price_url = price_url.replace('bmw-2_series', 'bmw-230')
    price_url = price_url.replace('lexus-es', 'lexus-es_350')
    driver.get(price_url)
    time.sleep(1)
    vehicle_prices.append([price_urls[price_index], price_url, 
                           driver.find_element_by_xpath('/html/body/section/div[2]/section[1]/div[2]/div[1]/div').text])
    print(str(price_index) + ': ' + price_url)
    price_index+=1

241: https://www.cars.com/research/mini-clubman-2019/
242: https://www.cars.com/research/mercedes_benz-glc_300-2019/
243: https://www.cars.com/research/subaru-brz-2019/
244: https://www.cars.com/research/ford-ranger-2019/
245: https://www.cars.com/research/kia-niro_ev-2019/
246: https://www.cars.com/research/toyota-sienna-2019/
247: https://www.cars.com/research/volvo-s60-2019/
248: https://www.cars.com/research/nissan-leaf-2019/
249: https://www.cars.com/research/nissan-gt_r-2019/
250: https://www.cars.com/research/mitsubishi-mirage-2019/
251: https://www.cars.com/research/mercedes_benz-gls_450-2019/
252: https://www.cars.com/research/honda-odyssey-2019/
253: https://www.cars.com/research/kia-cadenza-2019/
254: https://www.cars.com/research/ford-transit_connect-2019/
255: https://www.cars.com/research/honda-civic-2019/
256: https://www.cars.com/research/kia-niro-2019/
257: https://www.cars.com/research/toyota-prius-2019/
258: https://www.cars.com/research/nissan-altima-2019/
259: http

Add scraped price data to price_models_df

In [22]:
price_models_df[['URL', 'Scrape URL', 'Price']] = pd.DataFrame(vehicle_prices)
price_models_df.head()

,Unnamed: 0,Model,Reliability,Reliability.1,Acceleration,Acceleration.1,Braking,Braking.1,Ride,Ride.1,...,Max. Load,Max. Load.1,Brand,Unit,url,scrape url,price,URL,Scrape URL,Price
0,0,volvo-v60,Reliability,1 / 5,Acceleration,NaN,Braking,NaN,Ride,NaN,...,Max. Load,NaN,volvo,v60,https://www.cars.com/research/volvo-v60-2019/,https://www.cars.com/research/volvo-v60-2019/,"$38,900",https://www.cars.com/research/volvo-v60-2019/,https://www.cars.com/research/volvo-v60-2019/,"$38,900"
1,1,fiat-500l,Reliability,NaN,Acceleration,3 / 5,Braking,4 / 5,Ride,2 / 5,...,Max. Load,860 lb.,fiat,500l,https://www.cars.com/research/fiat-500l-2019/,https://www.cars.com/research/fiat-500l-2019/,"$22,160",https://www.cars.com/research/fiat-500l-2019/,https://www.cars.com/research/fiat-500l-2019/,"$22,160"
2,2,jaguar-xf,Reliability,NaN,Acceleration,5 / 5,Braking,5 / 5,Ride,5 / 5,...,Max. Load,960 lb.,jaguar,xf,https://www.cars.com/research/jaguar-xf-2019/,https://www.cars.com/research/jaguar-xf-2019/,"$49,965",https://www.cars.com/research/jaguar-xf-2019/,https://www.cars.com/research/jaguar-xf-2019/,"$49,965"
3,3,audi-tt,Reliability,NaN,Acceleration,5 / 5,Braking,5 / 5,Ride,3 / 5,...,Max. Load,770 lb.,audi,tt,https://www.cars.com/research/audi-tt-2019/,https://www.cars.com/research/audi-tt-2019/,"$44,900",https://www.cars.com/research/audi-tt-2019/,https://www.cars.com/research/audi-tt-2019/,"$44,900"
4,4,toyota-mirai,Reliability,NaN,Acceleration,3 / 5,Braking,3 / 5,Ride,4 / 5,...,Max. Load,690 lb.,toyota,mirai,https://www.cars.com/research/toyota-mirai-2019/,https://www.cars.com/research/toyota-mirai-2019/,"$58,500",https://www.cars.com/research/toyota-mirai-2019/,https://www.cars.com/research/toyota-mirai-2019/,"$58,500"


Create prices_df with only relevant price data

In [23]:
prices_df = price_models_df[['Model', 'Brand', 'Unit', 'URL', 'Scrape URL', 'Price']]
prices_df.head()

,Model,Brand,Unit,URL,Scrape URL,Price
0,volvo-v60,volvo,v60,https://www.cars.com/research/volvo-v60-2019/,https://www.cars.com/research/volvo-v60-2019/,"$38,900"
1,fiat-500l,fiat,500l,https://www.cars.com/research/fiat-500l-2019/,https://www.cars.com/research/fiat-500l-2019/,"$22,160"
2,jaguar-xf,jaguar,xf,https://www.cars.com/research/jaguar-xf-2019/,https://www.cars.com/research/jaguar-xf-2019/,"$49,965"
3,audi-tt,audi,tt,https://www.cars.com/research/audi-tt-2019/,https://www.cars.com/research/audi-tt-2019/,"$44,900"
4,toyota-mirai,toyota,mirai,https://www.cars.com/research/toyota-mirai-2019/,https://www.cars.com/research/toyota-mirai-2019/,"$58,500"


Save prices_df to csv file

In [24]:
prices_df.to_csv('Data/Prices.csv', index=False)

# Scrape Sales by Brand

Get html for goodcarbadcar webpage with sales by brand

In [7]:
driver.get('https://www.goodcarbadcar.net/2019-u-s-auto-sales-figures-by-brand/')

Extract sales data by brand from goodcarbadcar html

In [34]:
sales_by_brand = []
soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find('table', {'id' : 'table_1'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    row_data = []
    for row_datapoint in row.find_all('td'):
        row_data.append(row_datapoint.text)
    sales_by_brand.append([row_data[0], row_data[4]])
sales_by_brand[0:5]

[['Acura', '157,385'],
 ['Alfa Romeo', '18,294'],
 ['Audi', '224,111'],
 ['BMW', '324,825'],
 ['Buick', '206,926']]

Save sales_by_brand data to df

In [33]:
brand_sales_df = pd.DataFrame(sales_by_brand, columns=['Brand', 'Sales'])
brand_sales_df.head()

,Brand,Sales
0,Acura,"157,385"
1,Alfa Romeo,"18,294"
2,Audi,"224,111"
3,BMW,"324,825"
4,Buick,"206,926"


Save brand_sales_df to csv file

In [36]:
brand_sales_df.to_csv('Data/Brand_Sales.csv', index=False)